# Project IO

# MDF data pulls

In [1]:
import maad
maad.__version__

'1.7.25'

In [2]:
!git clone https://github.com/intel-sandbox/golden_fleece.git golden_fleece

fatal: destination path 'golden_fleece' already exists and is not an empty directory.


In [3]:
import golden_fleece as gf

In [4]:
import pandas as pd

In [5]:
r = pd.read_pickle('rebates_nov2_5Y')

In [6]:
r = maad.csar_cleanup(r)

In [7]:
r.shape

(619349, 62)

In [8]:
ix = r.CUSTOMER.dropna()[r.CUSTOMER.dropna().str.contains('hew', case = False)].index
#r.loc[ix]

In [9]:
r.loc[ix].DEAL_ID.dropna().unique()

array([250628., 719301., 250627., ..., 658498., 658509., 206646.])

In [10]:
dids = [int(x) for x in list(r.loc[ix].DEAL_ID.dropna().unique())]

In [11]:
len(dids)

1902

In [12]:
sql = '''
select DISTINCT  deal.[deal id]
 ,deal.[Tracker Number]
 ,cust.cust_nm as customer
 ,cast(deal.[deal start date] as datetime) as deal_start_date
 ,deal.[deal end date]
 ,deal.[deal stage]
 ,deal.[deal type]
 ,deal.geo
 ,deal.[market segment]
 ,deal.[End CustomerRetailer] as END_CUSTOMER
 ,deal.agreement
 ,deal.[Ceiling Volume]
 ,deal.[contract#] as contract
 ,deal.[deal description]
 ,deal.MSP
, deal.[Division Approver]
 ,deal.[Last Active Date]
, deal.[Server Deal Type]
 ,deal.[Division Approved Date]
 ,deal.[Division First Approved Stage Date]
,deal.[AdditiveStandalone]
,deal.[Expire Deal Flag]
,deal.[Overarching Maximum Volume]
,deal.[deal type]
,deal.[rebate type]
,deal.[Rebate Payment 609 Credit Amt]
,deal.[Rebate Payment 609 Debit Amt]
,deal.[Rebate Payment 609 Credit Volume]
,deal.[Rebate Payment 609 Debit Volume]

 
 from VW_RPT_DEAL_FACT deal inner join VW_RPT_DIM_CUST cust on 
 deal.[customer name ID] = cust.cust_nm_sid and
 deal.[customer id] = cust.cust_div_nm_sid
where 1 = 1 and cust.cust_nm = 'HPI' and deal.[rebate type] like '%MDF%'
'''

In [13]:
df = maad.get_data(sql, db = 'mydeals')

In [14]:
df.REBATE_TYPE.unique()

array(['MDF/NRE ACCRUAL', 'MDF', 'MDF SPIF/PER UNIT ACTIVITY',
       'MDF ACCRUAL', 'MDF/NRE LUMP-SUM BUDGET'], dtype=object)

In [15]:
df.remove_empty().shape

(196, 27)

In [16]:
dx = list(df.DEAL_ID.unique())

In [17]:
df = df.drop_duplicates(subset = 'DEAL_ID')

In [18]:
merge = pd.merge(df, r, on = 'DEAL_ID')

In [19]:
dy = [657997, 657998, 657999]

In [20]:
merge['NET_CLAIMED_AMT'] = (merge.REBATE_PAYMENT_609_CREDIT_AMT - merge.REBATE_PAYMENT_609_DEBIT_AMT)/1E6

In [21]:
mpivot = merge.query('DEAL_ID == @dx').pivot_table(index = ['YEAR'], columns = ['REBATE_TYPE', 'RELEVANT_YEAR', 'DEAL_ID'], aggfunc = 'sum',values = "AMOUNT", margins = True)/1E6

In [22]:
mpivot.to_clipboard()

In [23]:
df.query('DEAL_ID == 544415')

,DEAL_ID,TRACKER_NUMBER,CUSTOMER,DEAL_START_DATE,DEAL_END_DATE,DEAL_STAGE,DEAL_TYPE,GEO,MARKET_SEGMENT,END_CUSTOMER,...,DIVISION_FIRST_APPROVED_STAGE_DATE,ADDITIVESTANDALONE,EXPIRE_DEAL_FLAG,OVERARCHING_MAXIMUM_VOLUME,DEAL_TYPE,REBATE_TYPE,REBATE_PAYMENT_609_CREDIT_AMT,REBATE_PAYMENT_609_DEBIT_AMT,REBATE_PAYMENT_609_CREDIT_VOLUME,REBATE_PAYMENT_609_DEBIT_VOLUME
15,544415,PHP-544415-M-D,HPI,2019-01-01,2019-12-31,Active,PROGRAM,WW,"Consumer Retail Pull, Corp",NaN,...,2019-04-22,Additive,Yes,NaN,PROGRAM,MDF,1.186162e+08,162060.0,60.0,2.0


In [24]:
r.query('DEAL_ID == 544415').to_clipboard()

In [25]:
r.query('DEAL_ID == @dx').query('DEAL_ID == 600955')

,DEAL_QTY,RATE,ORIG_SLD_CUST_ID,UNIT_QTY,UNIT_RATE,TRACKER_BASE,FNL_APRV_DTM,ISTRACKER,CUSTOMER,DEAL_ID,...,MM,L2_TXT,CSAR_ID,MARKETING_NAME,CUST_PO_DT,UPLD_NM,SLS_BLLG_DOC_LINE_NBR,QTY,RELEVANT_YEAR,YEAR
139758,NaN,2028379.19,6004760766,1.0,2028379.19,PHP-600955-M,2020-04-15 16:58:08.217,True,HP International Pte. Ltd.,600955.0,...,892990,NaN,817336,NaN,NaT,NonInvBulkUpload,NaN,1,2020,2020
139763,NaN,1484466.40,6004760766,1.0,1484466.40,PHP-600955-M,2020-04-15 16:58:08.217,True,HP International Pte. Ltd.,600955.0,...,892990,NaN,817341,NaN,NaT,NonInvBulkUpload,NaN,1,2020,2020
155488,NaN,4348780.50,6004760766,1.0,4348780.50,PHP-600955-M,2020-07-23 13:09:42.420,True,HP International Pte. Ltd.,600955.0,...,892990,NaN,836646,NaN,NaT,NonInvBulkUpload,NaN,1,2020,2020
155489,NaN,1644106.23,6004760766,1.0,1644106.23,PHP-600955-M,2020-07-23 13:09:42.420,True,HP International Pte. Ltd.,600955.0,...,892990,NaN,836647,NaN,NaT,NonInvBulkUpload,NaN,1,2020,2020
155495,NaN,3751826.00,6004760766,1.0,3751826.00,PHP-600955-M,2020-07-23 13:09:42.420,True,HP International Pte. Ltd.,600955.0,...,892990,NaN,836653,NaN,NaT,NonInvBulkUpload,NaN,1,2020,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299874,NaN,50000.00,6100212581,1.0,50000.00,PHP-600955-M,2022-07-14 14:19:16.250,True,HP Inc.,600955.0,...,892990,NaN,994369,NaN,NaT,NonInvBulkUpload,NaN,1,2021,2022
299875,NaN,826085.00,6100212581,1.0,826085.00,PHP-600955-M,2022-07-14 14:19:16.250,True,HP Inc.,600955.0,...,892990,NaN,994370,NaN,NaT,NonInvBulkUpload,NaN,1,2020,2022
299876,NaN,420989.00,6100212581,1.0,420989.00,PHP-600955-M,2022-07-14 14:19:16.250,True,HP Inc.,600955.0,...,892990,NaN,994371,NaN,NaT,NonInvBulkUpload,NaN,1,2020,2022
2536,1.0,105000.00,6100212581,1.0,105000.00,PHP-600955-M,2022-10-25 15:39:41.953,True,NaN,600955.0,...,892990,NaN,1027540,NaN,NaT,NonInvBulkUpload,NaN,1,2020,2022


In [26]:
sql = '''
select DISTINCT  *

 
 from VW_RPT_DEAL_FACT deal inner join VW_RPT_DIM_CUST cust on 
 deal.[customer name ID] = cust.cust_nm_sid and
 deal.[customer id] = cust.cust_div_nm_sid
where 1 = 1  and deal.[deal id] = 600955
'''

In [27]:
df = maad.get_data(sql, db = 'mydeals', clean = False)

In [28]:
df.shape

(32, 149)

In [29]:
df.columns

Index(['Deal ID', 'OBJ_TYPE_SID', 'Deal Type', 'Customer ID',
       'Customer Name ID', 'Ceiling Volume', 'NOTES', 'Consumption Reason',
       'Deal Description', 'Deal End Date',
       ...
       'CUST_NM_SID', 'CUST_SRCH_NM', 'GDM_CUST_DIV_NM', 'GDM_SLD_TO_CUST_NM',
       'HOST_GEO', 'INDUS_KEY_CD', 'PRC_GRP_CD', 'SLS_ORG_CD', 'SOLD_TO_ID',
       'SOLD_TO_ID_SID'],
      dtype='object', length=149)

In [30]:
df.drop_duplicates().shape

(32, 149)

In [31]:
df.DEAL_TYPE

AttributeError: 'DataFrame' object has no attribute 'DEAL_TYPE'

In [34]:
df.to_clipboard()

In [33]:
df.to_excel()

TypeError: to_excel() missing 1 required positional argument: 'excel_writer'